## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_1 = pd.read_csv("data/"+'gcc6'+"_1_.csv", index_col=0)
core6_1 = pd.read_csv("data/"+"core_gcc6_1_.csv", index_col=0)

# 형태 출력
print(bin6_1.shape, core6_1.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_1.reset_index(inplace=True, drop=True)
core6_1.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
bin6_1.head()
core6_1.head()

(10478628, 2) (3455067, 2)
reset_index 완료
input data shape


,bin,label
0,86.0,1
1,83.0,0
2,131.0,0
3,236.0,0
4,4.0,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_1['bin'].unique()))
print(len(core6_1['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_1['label'].value_counts())
print(core6_1['label'].value_counts())

256
256
0    10452226
1       26402
Name: label, dtype: int64
0    3442761
1      12306
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_1[bin6_1['label']==1].index  # 407, 474 ...
idx_core = core6_1[core6_1['label']==1].index  # 407, 474 ...

ls_bin = list(idx_bin)
ls_core = list(idx_core)

# 최종 뽑을 행에 대한 index
ls_idx_bin, ls_idx_core = [], [] 

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능
    ls_idx_core.extend(list(idx_core + k))

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬
ls_idx_core.sort() # 인덱스 정렬


# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_1), ls_idx_bin))
ls_idx_core = list(filter(lambda x: x<len(core6_1), ls_idx_core))
print(len(ls_idx_bin), len(ls_idx_core))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
sub_core = len(ls_idx_core)%(right_idx)
print('나머지', sub_bin, sub_core)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
ls_idx_core = ls_idx_core[:len(ls_idx_core)-sub_core]
print('최종 길이', len(ls_idx_bin), len(ls_idx_core))

print('bin6_1', len(ls_idx_bin), len(ls_idx_core))

# loc 로 수정필요
bin6_1_Ngram = bin6_1.loc[ls_idx_bin,:].copy()
core6_1_Ngram = core6_1.loc[ls_idx_core,:].copy()

844864 393792
나머지 0 0
최종 길이 844864 393792
bin6_1 844864 393792


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_1_Ngram)/right_idx
count_bin = 0

goal_core = len(core6_1_Ngram)/right_idx
count_core = 0

print(goal_bin, goal_core)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_1.columns)
d_core = pd.DataFrame(columns = core6_1.columns)

binutils_df, coreutils_df = [], []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin) and (count_core == goal_core):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_1)-right_idx)
    random_idx_core = np.random.randint(len(core6_1)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin, random_idx_core)

    df_bin = bin6_1[random_idx_bin : random_idx_bin + right_idx]
    df_core = core6_1[random_idx_core : random_idx_core + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1
        
    if (1 not in df_core['label']) and count_core <goal_core:
        coreutils_df.append(df_core)
        count_core+=1

print('완료')
print(len(binutils_df), len(coreutils_df))

26402.0 12306.0
0 7626648 2109702
1000 6805824 2568435
2000 3835004 3431966
3000 5452552 2546290
4000 5940114 3155481
5000 11503 2381401
6000 4390287 105792
7000 5081187 2722387
8000 2950946 2726229
9000 4570309 948021
10000 4185209 2180794
11000 7883716 2843255
12000 1040595 2287498
13000 5149508 1189446
14000 988752 463391
15000 2660349 2892604
16000 6157679 2648700
17000 2186344 289165
18000 805306 2335491
19000 4722568 423175
20000 2453027 3190478
21000 6635030 1086016
22000 6518680 1651607
23000 7414156 3014843
24000 3045985 2590895
25000 10309810 1149700
26000 9857865 1414883
완료
26402 12306


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df) + len(coreutils_df))
print(bin6_1['label'].value_counts()[1] + core6_1['label'].value_counts()[1])

38708
38708


In [6]:
f_bin_data = pd.concat(binutils_df)
f_core_data = pd.concat(coreutils_df)
f_data = pd.concat([f_bin_data, f_core_data], axis=0)
f_data.shape

(1238656, 2)

## (5) False Data + True Data 합치기

In [7]:
final = pd.concat([f_data, bin6_1_Ngram])
final.shape

(2083520, 2)

## (6) one hot encoding

In [8]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_1_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_1_onehot_Ngram = pd.concat([final['label'], bc6_1_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_1_onehot_Ngram.shape)

원핫인코딩완료
(2083520, 257)


In [9]:
# 훈련 데이터, 훈련 라벨
x_bc6_1 = bc6_1_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_1 = bc6_1_onehot_Ngram['label'].to_numpy()
print(x_bc6_1.shape, x_bc6_1.shape)

x_bc6_1 = x_bc6_1.reshape(-1, right_idx, x_bc6_1.shape[1])
y_bc6_1 = y_bc6_1.reshape(-1, right_idx, 1)

print(x_bc6_1.shape, y_bc6_1.shape)

(2083520, 256) (2083520, 256)
(65110, 32, 256) (65110, 32, 1)


In [10]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_1.shape[0])

x_bc6_1 = x_bc6_1[p]
y_bc6_1 = y_bc6_1[p]

print(x_bc6_1.shape, y_bc6_1.shape)

(65110, 32, 256) (65110, 32, 1)


In [11]:
import tensorflow as tf 

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

## (7) 모델

In [12]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [13]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_1, y_bc6_1):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_1[train],
               y_bc6_1[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_1[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_1[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 22s 373us/sample - loss: 0.0161 - accuracy: 0.9957
Epoch 2/10
58599/58599 [==============================] - 16s 274us/sample - loss: 0.0033 - accuracy: 0.9990
Epoch 3/10
58599/58599 [==============================] - 15s 249us/sample - loss: 0.0021 - accuracy: 0.9993
Epoch 4/

accuracy_score 0.9996544309629857
recall_score 0.9937304075235109
precision_score 0.9858637263217416
f1_score 0.9897814362759012
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 21s 353us/sample - loss: 5.5956e-04 - accuracy: 0.9998
Epoch 2/10
58599/58599 [==============================] - 17s 294us/sample - loss: 4.8736e-

accuracy_score 0.9998416141913684
recall_score 0.9940880874963051
precision_score 0.996149289099526
f1_score 0.9951176209498447
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 20s 340us/sample - loss: 2.2946e-04 - accuracy: 0.9999
Epoch 2/10
58599/58599 [==============================] - 17s 292us/sample - loss: 2.0069e-0

accuracy_score 0.9999280064506221
recall_score 0.9961674528301887
precision_score 0.9994084590357882
f1_score 0.9977853240809095
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 20s 344us/sample - loss: 1.2179e-04 - accuracy: 1.0000
Epoch 2/10
58599/58599 [==============================] - 17s 298us/sample - loss: 9.3200e-

accuracy_score 0.9999568038703732
recall_score 0.9985586624387431
precision_score 0.9988465974625144
f1_score 0.9987026091970592
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 20s 346us/sample - loss: 6.2933e-05 - accuracy: 1.0000
Epoch 2/10
58599/58599 [==============================] - 17s 291us/sample - loss: 4.7284e-

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 19s 324us/sample - loss: 2.4704e-05 - accuracy: 1.0000
Epoch 2/10
58599/58599 [==============================] - 18s 301us/sample - loss: 2.4684e-05 - accuracy: 1.0000
Epoch 3/10
58599/58599 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 20s 344us/sample - loss: 2.4679e-05 - accuracy: 1.0000
Epoch 2/10
58599/58599 [==============================] - 17s 298us/sample - loss: 2.4678e-05 - accuracy: 1.0000
Epoch 3/10
58599/58599 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 20s 346us/sample - loss: 1.6452e-05 - accuracy: 1.0000
Epoch 2/10
58599/58599 [==============================] - 18s 299us/sample - loss: 1.6452e-05 - accuracy: 1.0000
Epoch 3/10
58599/58599 [==============

accuracy_score 0.9999952004300414
recall_score 0.9997131382673551
precision_score 1.0
f1_score 0.999856548558313
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 58599 samples
Epoch 1/10
58599/58599 [==============================] - 21s 354us/sample - loss: 2.4678e-05 - accuracy: 1.0000
Epoch 2/10
58599/58599 [==============================] - 20s 337us/sample - loss: 2.4678e-05 - accuracy: 1

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9996544309629857, 0.9998416141913684, 0.9999280064506221, 0.9999568038703732, 1.0, 1.0, 1.0, 1.0, 0.9999952004300414, 1.0]

K-fold cross validation Recall: [0.9937304075235109, 0.9940880874963051, 0.9961674528301887, 0.9985586624387431, 1.0, 1.0, 1.0, 1.0, 0.9997131382673551, 1.0]

K-fold cross validation Precision: [0.9858637263217416, 0.996149289099526, 0.9994084590357882, 0.9988465974625144, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9897814362759012, 0.9951176209498447, 0.9977853240809095, 0.9987026091970592, 1.0, 1.0, 1.0, 1.0, 0.999856548558313, 1.0]


## (9) 평가지표

In [14]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.999937605590539
10-Fold Cross_validation. Recall : 0.9982257748556103
10-Fold Cross_validation. Precision : 0.9980268071919571
10-Fold Cross_validation. F1-Score : 0.9981243539062028
